Data preprocessing 

In [1]:
import os
import numpy as np

data_path="C:\\Users\\ASHAN\\Desktop\\Dataset_01"
catagories=os.listdir(data_path)
print(catagories)
labels=np.arange(len(catagories))
print(labels)
label_dict=dict(zip(catagories,labels))
print(label_dict)


['cats', 'Dogs']
[0 1]
{'cats': np.int64(0), 'Dogs': np.int64(1)}


In [11]:
import cv2
img_size=50


for catagory in catagories:
    
    imgs_path=os.path.join(data_path,catagory)
    #print(img_path)
    img_names=os.listdir(imgs_path)
    
    
    for img_name in img_names:
        img_path=os.path.join(imgs_path,img_name)
        print(img_path)
        try:
            img=cv2.imread(img_path)
            img=cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
            img=cv2.resize(img,(img_size,img_size))
        except Exception as e :
            print(img_path,e)

    

C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.0.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.1.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.10.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.100.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.101.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.1018.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.102.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.1022.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.1024.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.1029.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.103.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.104.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.105.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.106.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.107.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.108.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.109.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.11.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.110.jpg
C:\Users\ASHAN\Desktop\Dataset_01

C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.321.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.322.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.323.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.324.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.325.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.326.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.327.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.328.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.329.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.33.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.330.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.331.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.332.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.333.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.334.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.335.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.336.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.337.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.338.jpg
C:\Users\ASHAN\Desktop\Dataset_0

C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.93.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.930.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.933.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.94.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.946.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.95.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.955.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.96.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.960.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.961.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.962.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.965.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.97.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.974.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.977.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.98.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.99.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.992.jpg
C:\Users\ASHAN\Desktop\Dataset_01\cats\cat.996.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs

C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.303.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.304.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.305.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.306.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.307.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.308.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.309.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.31.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.310.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.311.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.312.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.313.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.314.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.315.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.316.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.317.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.318.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.319.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.32.jpg
C:\Users\ASHAN\Desktop\Dataset_01

C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.793.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.794.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.798.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.8.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.80.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.801.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.81.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.810.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.817.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.818.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.819.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.82.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.824.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.83.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.830.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.836.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.837.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.84.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs\dog.843.jpg
C:\Users\ASHAN\Desktop\Dataset_01\Dogs